In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap

In [18]:
df_train = pd.read_parquet("../data/interim/train_final_advanced_features.parquet")

In [19]:
TARGET = "TARGET"
y = df_train[TARGET]
X = df_train.drop(columns=[TARGET])

In [20]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 205


In [21]:
num_imputer = SimpleImputer(strategy="median") #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [22]:
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)


In [23]:
print(X.isna().sum().sum())  # must be 0
print(X.shape)

0
(307511, 319)


In [24]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42)

In [25]:
'''
This library needs sklearn 1.3.2
and imblearn 0.11.0
'''

'''
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline


# Combine over and under sampling
over = SMOTE(sampling_strategy=0.3)
under = RandomUnderSampler(sampling_strategy=0.7)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)
X_train, y_train = pipeline.fit_resample(X_train, y_train)
'''

"\nfrom imblearn.over_sampling import SMOTE\nfrom imblearn.under_sampling import RandomUnderSampler\nfrom imblearn.pipeline import Pipeline as ImbPipeline\n\n\n# Combine over and under sampling\nover = SMOTE(sampling_strategy=0.3)\nunder = RandomUnderSampler(sampling_strategy=0.7)\nsteps = [('over', over), ('under', under)]\npipeline = ImbPipeline(steps=steps)\nX_train, y_train = pipeline.fit_resample(X_train, y_train)\n"

In [26]:
X_train.shape


(246008, 319)

In [27]:
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
optimal_scale = neg_count / pos_count

In [28]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}


xgb_model = XGBClassifier(**params, n_jobs=-1, random_state=42,  early_stopping_rounds=300)

In [29]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.70399
[1]	validation_0-auc:0.71348
[2]	validation_0-auc:0.71987
[3]	validation_0-auc:0.72760
[4]	validation_0-auc:0.72773
[5]	validation_0-auc:0.72811
[6]	validation_0-auc:0.72773
[7]	validation_0-auc:0.72749
[8]	validation_0-auc:0.72819
[9]	validation_0-auc:0.72828
[10]	validation_0-auc:0.72813
[11]	validation_0-auc:0.72933
[12]	validation_0-auc:0.72975
[13]	validation_0-auc:0.72969
[14]	validation_0-auc:0.73058
[15]	validation_0-auc:0.73260
[16]	validation_0-auc:0.73210
[17]	validation_0-auc:0.73289
[18]	validation_0-auc:0.73379
[19]	validation_0-auc:0.73469
[20]	validation_0-auc:0.73563
[21]	validation_0-auc:0.73662
[22]	validation_0-auc:0.73662
[23]	validation_0-auc:0.73708
[24]	validation_0-auc:0.73689
[25]	validation_0-auc:0.73653
[26]	validation_0-auc:0.73711
[27]	validation_0-auc:0.73691
[28]	validation_0-auc:0.73759
[29]	validation_0-auc:0.73739
[30]	validation_0-auc:0.73766
[31]	validation_0-auc:0.73773
[32]	validation_0-auc:0.73808
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.75, device='cuda', early_stopping_rounds=300,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=2, max_depth=5,
              max_leaves=31, min_child_weight=15, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=6000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [30]:

# ==========================================
# CONFIGURACIÓN 1: ULTRA CONSERVADORA
# Mayor generalización, menos overfit
# ==========================================
params_ultra_conservative = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Learning muy bajo para aprendizaje gradual
    'learning_rate': 0.005,
    'n_estimators': 10000,

    # Árboles muy simples
    'max_depth': 3,
    'min_child_weight': 30,
    'max_leaves': 15,

    # Sampling muy agresivo para diversidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    # Balance de clases
    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 2.0,      # L1
    'reg_lambda': 5.0,     # L2
    'gamma': 3.0,          # Min loss reduction
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 2: CONSERVADORA BALANCEADA
# Buen equilibrio generalización/performance
# ==========================================
params_conservative_balanced = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7500,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 20,

    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'colsample_bylevel': 0.75,
    'colsample_bynode': 0.8,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 3.0,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 3: MODERADA
# Tu configuración actual mejorada
# ==========================================
params_moderate = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,
    'max_leaves': 31,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 0.5,
    'reg_lambda': 2.5,
    'gamma': 1.5,
    'max_delta_step': 2,
}

# ==========================================
# CONFIGURACIÓN 4: AGRESIVA PARA RECALL
# Maximiza detección de defaults
# ==========================================
params_high_recall = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,
    'n_estimators': 5000,

    # Árboles más profundos para capturar patrones complejos
    'max_depth': 6,
    'min_child_weight': 8,
    'max_leaves': 50,

    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.85,
    'colsample_bynode': 0.9,

    # Más peso a clase positiva
    'scale_pos_weight': optimal_scale * 1.3,

    # Regularización más suave
    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.5,
    'max_delta_step': 3,
}

# ==========================================
# CONFIGURACIÓN 5: AGRESIVA PARA PRECISION
# Minimiza falsos positivos
# ==========================================
params_high_precision = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 4309,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.8,
    'colsample_bynode': 0.85,

    # Menos peso a positivos para ser más selectivo
    'scale_pos_weight': optimal_scale * 0.8,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 6: DART (Dropout Trees)
# Previene overfit con dropout en árboles
# ==========================================
params_dart = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',
    'booster': 'dart',  # Usa DART en lugar de gbtree

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # DART específicos
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'rate_drop': 0.1,
    'skip_drop': 0.5,

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 7: FAST & LIGHT
# Rápido para iteraciones, menor memoria
# ==========================================
params_fast = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Más rápido pero efectivo
    'learning_rate': 0.02,
    'n_estimators': 3000,

    'max_depth': 4,
    'min_child_weight': 20,
    'max_leaves': 15,
    'max_bin': 128,  # Reduce bins para velocidad

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.0,
    'reg_lambda': 2.0,
    'gamma': 1.5,
}

# ==========================================
# CONFIGURACIÓN 8: DEEP TREES
# Árboles profundos con fuerte regularización
# ==========================================
params_deep = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 7000,

    # Muy profundos
    'max_depth': 8,
    'min_child_weight': 10,
    'max_leaves': 100,

    # Sampling fuerte para compensar profundidad
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'colsample_bylevel': 0.6,
    'colsample_bynode': 0.6,

    'scale_pos_weight': optimal_scale,

    # Regularización muy fuerte
    'reg_alpha': 3.0,
    'reg_lambda': 5.0,
    'gamma': 4.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIGURACIÓN 9: MONOTONE CONSTRAINTS
# Fuerza relaciones lógicas en features
# ==========================================
params_monotone = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Monotone constraints (ajustar según tus features)
    # +1: a mayor valor, mayor probabilidad de default
    # -1: a mayor valor, menor probabilidad de default
    # 0: sin restricción
    # Ejemplo para primeras 10 features (ajustar según tu caso)
    'monotone_constraints': '(0,0,0,0,0,1,1,-1,1,0)',  # Ejemplo

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 10: INTERACTION CONSTRAINTS
# Limita qué features pueden interactuar
# ==========================================
params_interaction = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.01,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 15,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale,

    # Permite interacciones solo dentro de grupos
    # Ejemplo: [[0,1,2], [3,4,5]] significa que features 0,1,2 pueden interactuar entre sí
    # y 3,4,5 entre sí, pero no entre grupos
    # 'interaction_constraints': [[0,1,2,3], [4,5,6,7], [8,9,10]],  # Ajustar a tus features

    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}

# ==========================================
# CONFIGURACIÓN 11: TWO-STAGE ENSEMBLE
# Entrena dos modelos complementarios
# ==========================================
params_ensemble_stage1 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en generalización
    'learning_rate': 0.008,
    'n_estimators': 6000,

    'max_depth': 4,
    'min_child_weight': 25,

    'subsample': 0.7,
    'colsample_bytree': 0.7,

    'scale_pos_weight': optimal_scale,

    'reg_alpha': 1.5,
    'reg_lambda': 3.0,
    'gamma': 2.0,
}

params_ensemble_stage2 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'tree_method': 'hist',
    'device': 'cuda',

    # Enfocado en capturar residuos
    'learning_rate': 0.012,
    'n_estimators': 4000,

    'max_depth': 6,
    'min_child_weight': 10,

    'subsample': 0.8,
    'colsample_bytree': 0.8,

    'scale_pos_weight': optimal_scale * 1.2,

    'reg_alpha': 0.3,
    'reg_lambda': 1.5,
    'gamma': 0.8,
}

# =========================================
# BASADO EN EL MEJOR AUC 5
# =========================================

params_best_auc = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Ligeramente menor que 0.012
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 22,  # Entre 20 y 25
    'max_leaves': 28,

    'subsample': 0.75,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.85,  # Entre 0.8 y 1.0

    'reg_alpha': 0.7,
    'reg_lambda': 2.5,
    'gamma': 1.8,
    'max_delta_step': 1,
}

# ==========================================
# RESUMEN DE CONFIGURACIONES
# ==========================================
configs = {
    '1_ultra_conservative': params_ultra_conservative,
    '2_conservative_balanced': params_conservative_balanced,
    '3_moderate': params_moderate,
    '4_high_recall': params_high_recall,
    '5_high_precision': params_high_precision,
    #'6_dart': params_dart,         #Tarda como 3 horas en entrenar con gpu
    '7_fast': params_fast,
    '8_deep': params_deep,
    '9_monotone': params_monotone,
    '10_interaction': params_interaction,
    '11_ensemble_s1': params_ensemble_stage1,
    '11_ensemble_s2': params_ensemble_stage2,
    '12_best_auc': params_best_auc,
}

# ==========================================
# FUNCIÓN PARA ENTRENAR Y COMPARAR
# ==========================================
def train_and_evaluate(config_name, params, X_train, y_train, X_val, y_val):
    """
    Entrena un modelo con la configuración dada y evalúa
    """
    print(f"\n{'='*70}")
    print(f"Entrenando: {config_name}")
    print(f"{'='*70}")

    # Preparar parámetros
    train_params = params.copy()

    # Extraer n_estimators y early_stopping_rounds
    n_estimators = train_params.pop('n_estimators', 5000)
    early_stopping = min(500, n_estimators // 10)

    # Crear modelo
    model = XGBClassifier(
        **train_params,
        n_estimators=n_estimators,
        n_jobs=-1,
        random_state=42,
        early_stopping_rounds=early_stopping
    )

    # Entrenar
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=200
    )

    # Evaluar
    from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix

    y_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.5).astype(int)

    auc = roc_auc_score(y_val, y_proba)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
    fpr = fp / (fp + tn)

    print(f"\n{'='*70}")
    print(f"RESULTADOS: {config_name}")
    print(f"{'='*70}")
    print(f"AUC:       {auc:.5f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"FPR:       {fpr:.4f}")
    print(f"Best iteration: {model.best_iteration}")

    return {
        'config': config_name,
        'model': model,
        'auc': auc,
        'precision': precision,
        'recall': recall,
        'fpr': fpr,
        'best_iteration': model.best_iteration
    }

In [31]:
results = {}

for i, j in configs.items():
    results[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 1_ultra_conservative
[0]	validation_0-auc:0.68234
[200]	validation_0-auc:0.73439
[400]	validation_0-auc:0.74282
[600]	validation_0-auc:0.74884
[800]	validation_0-auc:0.75367
[1000]	validation_0-auc:0.75771
[1200]	validation_0-auc:0.76124
[1400]	validation_0-auc:0.76392
[1600]	validation_0-auc:0.76613
[1800]	validation_0-auc:0.76788
[2000]	validation_0-auc:0.76937
[2200]	validation_0-auc:0.77067
[2400]	validation_0-auc:0.77180
[2600]	validation_0-auc:0.77281
[2800]	validation_0-auc:0.77372
[3000]	validation_0-auc:0.77459
[3200]	validation_0-auc:0.77527
[3400]	validation_0-auc:0.77603
[3600]	validation_0-auc:0.77669
[3800]	validation_0-auc:0.77729
[4000]	validation_0-auc:0.77788
[4200]	validation_0-auc:0.77838
[4400]	validation_0-auc:0.77883
[4600]	validation_0-auc:0.77930
[4800]	validation_0-auc:0.77968
[5000]	validation_0-auc:0.78006
[5200]	validation_0-auc:0.78039
[5400]	validation_0-auc:0.78073
[5600]	validation_0-auc:0.78105
[5800]	validation_0-auc:0.78137
[6000]	valida

C:\Users\usuario\Desktop\4geeks\Final_Proyect_Credit_Default_Risk-main\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [20:59:53] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)



RESULTADOS: 1_ultra_conservative
AUC:       0.78511
Precision: 0.1851
Recall:    0.6967
FPR:       0.2694
Best iteration: 9988

Entrenando: 2_conservative_balanced
[0]	validation_0-auc:0.69413
[200]	validation_0-auc:0.74340
[400]	validation_0-auc:0.75625
[600]	validation_0-auc:0.76430
[800]	validation_0-auc:0.76919
[1000]	validation_0-auc:0.77233
[1200]	validation_0-auc:0.77462
[1400]	validation_0-auc:0.77652
[1600]	validation_0-auc:0.77813
[1800]	validation_0-auc:0.77929
[2000]	validation_0-auc:0.78045
[2200]	validation_0-auc:0.78134
[2400]	validation_0-auc:0.78216
[2600]	validation_0-auc:0.78280
[2800]	validation_0-auc:0.78336
[3000]	validation_0-auc:0.78382
[3200]	validation_0-auc:0.78431
[3400]	validation_0-auc:0.78463
[3600]	validation_0-auc:0.78503
[3800]	validation_0-auc:0.78527
[4000]	validation_0-auc:0.78560
[4200]	validation_0-auc:0.78583
[4400]	validation_0-auc:0.78606
[4600]	validation_0-auc:0.78627
[4800]	validation_0-auc:0.78643
[5000]	validation_0-auc:0.78667
[5200]	val

In [32]:
for key, d in results.items():
    print(key)
    print({k: d[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

1_ultra_conservative
{'auc': 0.7851112194787262, 'precision': 0.18508213387554176, 'recall': 0.6966767371601208, 'fpr': 0.26937634865046517, 'best_iteration': 9988}
2_conservative_balanced
{'auc': 0.7881102843182193, 'precision': 0.19519449466378958, 'recall': 0.6741188318227593, 'fpr': 0.24408362517245039, 'best_iteration': 7452}
3_moderate
{'auc': 0.7876043016029608, 'precision': 0.20367524651043667, 'recall': 0.6406847935548842, 'fpr': 0.21997594538186707, 'best_iteration': 5230}
4_high_recall
{'auc': 0.7857668560178778, 'precision': 0.18297224990819913, 'recall': 0.7025176233635448, 'fpr': 0.27547843927977644, 'best_iteration': 2207}
5_high_precision
{'auc': 0.788240261333384, 'precision': 0.22435452793834296, 'recall': 0.5863041289023162, 'fpr': 0.17800417418373482, 'best_iteration': 4264}
7_fast
{'auc': 0.7873548565951259, 'precision': 0.19618146785776844, 'recall': 0.676737160120846, 'fpr': 0.24349994693834234, 'best_iteration': 2992}
8_deep
{'auc': 0.7868749131001804, 'precisio

In [33]:
# ==========================================
# MICRO-OPTIMIZACIÓN DE 5_HIGH_PRECISION
# Exploración fina alrededor del mejor modelo
# ==========================================

# BASE DE REFERENCIA (5_high_precision - AUC: 0.78817)
# 'learning_rate': 0.012
# 'scale_pos_weight': optimal_scale * 0.80
# 'min_child_weight': 25
# 'max_leaves': 25
# 'reg_alpha': 0.8, 'reg_lambda': 2.5, 'gamma': 2.0

# ==========================================
# GRUPO 1: AJUSTE DE SCALE_POS_WEIGHT
# El hiperparámetro MÁS crítico según resultados
# ==========================================

# Config 13: Scale más bajo (más conservador)
params_13_scale_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.76,  # Entre 0.76-0.80

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 14: Scale ligeramente más alto
params_14_scale_med = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.82,  # Entre 0.80-0.85

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 2: AJUSTE DE MIN_CHILD_WEIGHT
# Segundo parámetro más sensible
# ==========================================

# Config 15: Más restrictivo (menos overfit)
params_15_mcw_high = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 28,  # Más alto que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 16: Menos restrictivo (más capacidad)
params_16_mcw_low = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 22,  # Más bajo que 25
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 3: AJUSTE DE REGULARIZACIÓN (L1, L2, GAMMA)
# Balance entre los 3 tipos de regularización
# ==========================================

# Config 17: Regularización más suave
params_17_reg_soft = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.6,   # Menos L1
    'reg_lambda': 2.2,  # Menos L2
    'gamma': 1.7,       # Menos gamma
    'max_delta_step': 1,
}

# Config 18: Regularización más fuerte
params_18_reg_strong = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1
    'reg_lambda': 2.8,  # Más L2
    'gamma': 2.3,       # Más gamma
    'max_delta_step': 1,
}

# Config 19: Balance L1 vs L2 diferente
params_19_reg_balance = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.0,   # Más L1 (feature selection)
    'reg_lambda': 2.0,  # Menos L2 (suavidad)
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 4: LEARNING RATE + MAX_LEAVES
# Combinaciones ajustadas
# ==========================================

# Config 20: Learning rate más lento, más árboles simples
params_20_slow_simple = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,  # Más lento
    'n_estimators': 7200,    # Más iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 22,  # Más simple

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 21: Learning rate más rápido, árboles ligeramente más complejos
params_21_fast_complex = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,  # Más rápido
    'n_estimators': 5500,    # Menos iteraciones

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,  # Más complejo

    'subsample': 0.75,
    'colsample_bytree': 0.75,
    'colsample_bylevel': 0.80,
    'colsample_bynode': 0.85,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 5: SAMPLING VARIATIONS
# Ajustes en subsample y colsample_*
# ==========================================

# Config 22: Más sampling (más diversidad)
params_22_high_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.78,           # Más alto
    'colsample_bytree': 0.78,    # Más alto
    'colsample_bylevel': 0.82,   # Más alto
    'colsample_bynode': 0.87,    # Más alto

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# Config 23: Menos sampling (más conservador)
params_23_low_sample = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 25,

    'subsample': 0.72,           # Más bajo
    'colsample_bytree': 0.72,    # Más bajo
    'colsample_bylevel': 0.78,   # Más bajo
    'colsample_bynode': 0.83,    # Más bajo

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 0.8,
    'reg_lambda': 2.5,
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# GRUPO 6: COMBINACIONES ÓPTIMAS
# Mejores combinaciones basadas en intuición
# ==========================================

# Config 24: Combinación conservadora óptima
params_24_ultra_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 5,
    'min_child_weight': 26,  # Ligeramente más restrictivo
    'max_leaves': 24,        # Ligeramente más simple

    'subsample': 0.76,
    'colsample_bytree': 0.76,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.78,  # Entre 0.76 y 0.80

    'reg_alpha': 0.85,  # Entre 0.8 y 0.9
    'reg_lambda': 2.6,  # Entre 2.5 y 2.7
    'gamma': 2.1,       # Entre 2.0 y 2.2
    'max_delta_step': 1,
}

# Config 25: Combinación agresiva óptima
params_25_balanced_optimal = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.0125,  # Entre 0.012 y 0.013
    'n_estimators': 5800,

    'max_depth': 5,
    'min_child_weight': 24,  # Ligeramente menos restrictivo
    'max_leaves': 26,        # Ligeramente más complejo

    'subsample': 0.77,
    'colsample_bytree': 0.77,
    'colsample_bylevel': 0.81,
    'colsample_bynode': 0.86,

    'scale_pos_weight': optimal_scale * 0.81,  # Entre 0.80 y 0.82

    'reg_alpha': 0.75,  # Menos L1
    'reg_lambda': 2.4,  # Menos L2
    'gamma': 1.9,       # Menos gamma
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO DE TODAS LAS CONFIGS
# ==========================================
all_fine_tuned_configs = {
    '13_scale_low': params_13_scale_low,
    '14_scale_med': params_14_scale_med,
    '15_mcw_high': params_15_mcw_high,
    '16_mcw_low': params_16_mcw_low,
    '17_reg_soft': params_17_reg_soft,
    '18_reg_strong': params_18_reg_strong,
    '19_reg_balance': params_19_reg_balance,
    '20_slow_simple': params_20_slow_simple,
    '21_fast_complex': params_21_fast_complex,
    '22_high_sample': params_22_high_sample,
    '23_low_sample': params_23_low_sample,
    '24_ultra_optimal': params_24_ultra_optimal,
    '25_balanced_optimal': params_25_balanced_optimal,
}



In [34]:
results_tuned = {}

for i, j in all_fine_tuned_configs.items():
    results_tuned[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 13_scale_low
[0]	validation_0-auc:0.69824
[200]	validation_0-auc:0.75472
[400]	validation_0-auc:0.76805
[600]	validation_0-auc:0.77398
[800]	validation_0-auc:0.77763
[1000]	validation_0-auc:0.77993
[1200]	validation_0-auc:0.78165
[1400]	validation_0-auc:0.78304
[1600]	validation_0-auc:0.78421
[1800]	validation_0-auc:0.78509
[2000]	validation_0-auc:0.78581
[2200]	validation_0-auc:0.78628
[2400]	validation_0-auc:0.78663
[2600]	validation_0-auc:0.78698
[2800]	validation_0-auc:0.78718
[3000]	validation_0-auc:0.78747
[3200]	validation_0-auc:0.78761
[3400]	validation_0-auc:0.78777
[3600]	validation_0-auc:0.78787
[3800]	validation_0-auc:0.78802
[4000]	validation_0-auc:0.78796
[4200]	validation_0-auc:0.78800
[4400]	validation_0-auc:0.78799
[4600]	validation_0-auc:0.78806
[4800]	validation_0-auc:0.78808
[5000]	validation_0-auc:0.78804
[5200]	validation_0-auc:0.78803
[5381]	validation_0-auc:0.78796

RESULTADOS: 13_scale_low
AUC:       0.78810
Precision: 0.2329
Recall:    0.5652
FPR:

In [35]:
for i, j in results_tuned.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

13_scale_low
{'auc': 0.7881017150119105, 'precision': 0.23294039515191764, 'recall': 0.5651560926485398, 'fpr': 0.1634299055502494, 'best_iteration': 4881}
14_scale_med
{'auc': 0.7877959576742172, 'precision': 0.22058055369002313, 'recall': 0.5953675730110776, 'fpr': 0.18474300470480032, 'best_iteration': 3988}
15_mcw_high
{'auc': 0.7881367421182421, 'precision': 0.22358570656787838, 'recall': 0.583484390735146, 'fpr': 0.17793342530687326, 'best_iteration': 4265}
16_mcw_low
{'auc': 0.7876286433489627, 'precision': 0.22469916692378897, 'recall': 0.5867069486404833, 'fpr': 0.17777424033393469, 'best_iteration': 4196}
17_reg_soft
{'auc': 0.7877643397660307, 'precision': 0.2235176804479558, 'recall': 0.586908358509567, 'fpr': 0.17904772011744313, 'best_iteration': 4126}
18_reg_strong
{'auc': 0.7882788828816466, 'precision': 0.2244850003916347, 'recall': 0.5772406847935548, 'fpr': 0.17512115745162546, 'best_iteration': 4805}
19_reg_balance
{'auc': 0.788649625164542, 'precision': 0.225908123

In [45]:
xgb_model = XGBClassifier(**params_19_reg_balance, n_jobs=-1, random_state=42)
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)

[0]	validation_0-auc:0.70100
[1]	validation_0-auc:0.71172
[2]	validation_0-auc:0.71733
[3]	validation_0-auc:0.72628
[4]	validation_0-auc:0.72609
[5]	validation_0-auc:0.72657
[6]	validation_0-auc:0.72574
[7]	validation_0-auc:0.72531
[8]	validation_0-auc:0.72555
[9]	validation_0-auc:0.72555
[10]	validation_0-auc:0.72516
[11]	validation_0-auc:0.72661
[12]	validation_0-auc:0.72687
[13]	validation_0-auc:0.72687
[14]	validation_0-auc:0.72804
[15]	validation_0-auc:0.73022
[16]	validation_0-auc:0.72984
[17]	validation_0-auc:0.73064
[18]	validation_0-auc:0.73150
[19]	validation_0-auc:0.73251
[20]	validation_0-auc:0.73368
[21]	validation_0-auc:0.73471
[22]	validation_0-auc:0.73474
[23]	validation_0-auc:0.73522
[24]	validation_0-auc:0.73498
[25]	validation_0-auc:0.73459
[26]	validation_0-auc:0.73520
[27]	validation_0-auc:0.73512
[28]	validation_0-auc:0.73561
[29]	validation_0-auc:0.73544
[30]	validation_0-auc:0.73583
[31]	validation_0-auc:0.73586
[32]	validation_0-auc:0.73613
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.85,
              colsample_bytree=0.75, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=2.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.012, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=5,
              max_leaves=25, min_child_weight=25, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=6000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [46]:
#optimal f1 threshold
from sklearn.metrics import f1_score
import numpy as np

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1): {optimal_threshold:.3f}")

Optimal Threshold (F1): 0.578


In [47]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve

y_val_pred = xgb_model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_val_pred)

adj_threshold = optimal_threshold

y_val_pred = (y_val_pred >= adj_threshold).astype(int)

report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

FPR: 0.1103
Precision (PPV): 0.2721
AUC validación: 0.78826
              precision    recall  f1-score   support

           0       0.95      0.89      0.92     56538
           1       0.27      0.47      0.34      4965

    accuracy                           0.86     61503
   macro avg       0.61      0.68      0.63     61503
weighted avg       0.90      0.86      0.87     61503



In [39]:
# ==========================================
# CONFIGURACIONES XGBOOST PARA 363 FEATURES
# Optimizadas para dataset con feature engineering avanzado
# ==========================================

# IMPORTANTE: Con 363 features (vs ~120 originales), necesitas:
# 1. Más regularización (para evitar overfit con tantas features)
# 2. Más colsample (sampling de features más agresivo)
# 3. Árboles potencialmente más profundos (para capturar interacciones)

# ==========================================
# CONFIG 26: HIGH PRECISION CON MUCHAS FEATURES
# Adaptación del mejor modelo (5_high_precision) para 363 features
# ==========================================
params_26_hp_many_features = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,  # +1 vs original (más features = más profundidad OK)
    'min_child_weight': 30,  # +5 (más conservador con muchas features)
    'max_leaves': 30,  # +5

    # ⚡ CRÍTICO: Sampling más agresivo con 363 features
    'subsample': 0.70,  # -0.05
    'colsample_bytree': 0.65,  # -0.10 (muy importante!)
    'colsample_bylevel': 0.70,  # -0.10
    'colsample_bynode': 0.75,  # -0.10

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización más fuerte
    'reg_alpha': 1.2,   # +0.4
    'reg_lambda': 3.5,  # +1.0
    'gamma': 2.5,       # +0.5
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 27: ULTRA REGULARIZADO PARA MUCHAS FEATURES
# Previene overfit agresivamente
# ==========================================
params_27_ultra_reg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.010,
    'n_estimators': 7000,

    'max_depth': 5,
    'min_child_weight': 35,
    'max_leaves': 25,

    # Feature sampling MUY agresivo
    'subsample': 0.65,
    'colsample_bytree': 0.60,
    'colsample_bylevel': 0.65,
    'colsample_bynode': 0.70,

    'scale_pos_weight': optimal_scale * 0.80,

    # Regularización extrema
    'reg_alpha': 2.0,
    'reg_lambda': 5.0,
    'gamma': 3.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 28: DEEP TREES CON FEATURE SELECTION
# Árboles profundos + sampling fuerte = explora interacciones
# ==========================================
params_28_deep_selection = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.008,
    'n_estimators': 8000,

    # Profundo para capturar interacciones complejas
    'max_depth': 7,
    'min_child_weight': 25,
    'max_leaves': 50,

    # Sampling agresivo compensa profundidad
    'subsample': 0.65,
    'colsample_bytree': 0.55,  # Muy bajo: cada árbol ve ~200 features
    'colsample_bylevel': 0.60,
    'colsample_bynode': 0.65,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.5,  # L1 alto para feature selection
    'reg_lambda': 4.0,
    'gamma': 3.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 29: BALANCED DIVERSITY
# Balance entre exploración y explotación
# ==========================================
params_29_balanced_diverse = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 28,
    'max_leaves': 35,

    'subsample': 0.68,
    'colsample_bytree': 0.62,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.5,
    'reg_lambda': 3.8,
    'gamma': 2.7,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 30: FEATURE SAMPLING EXTREMO
# Cada árbol ve solo ~30% de features (ensemble diverso)
# ==========================================
params_30_extreme_sampling = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.013,
    'n_estimators': 6000,

    'max_depth': 5,
    'min_child_weight': 25,
    'max_leaves': 28,

    # SAMPLING EXTREMO
    'subsample': 0.60,
    'colsample_bytree': 0.50,  # Solo 180 features por árbol
    'colsample_bylevel': 0.55,
    'colsample_bynode': 0.60,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.8,
    'reg_lambda': 3.0,
    'gamma': 2.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 31: L1 DOMINANT (Feature Selection)
# Alta regularización L1 para selección automática de features
# ==========================================
params_31_l1_dominant = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 30,
    'max_leaves': 32,

    'subsample': 0.70,
    'colsample_bytree': 0.65,
    'colsample_bylevel': 0.70,
    'colsample_bynode': 0.75,

    'scale_pos_weight': optimal_scale * 0.80,

    # L1 muy alto para feature selection
    'reg_alpha': 3.0,  # ⚡ Muy alto
    'reg_lambda': 2.0,  # L2 más bajo
    'gamma': 2.0,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 32: ADAPTIVE LEARNING
# Learning rate más alto con más regularización
# ==========================================
params_32_adaptive = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.015,  # Más rápido
    'n_estimators': 5000,

    'max_depth': 5,
    'min_child_weight': 32,
    'max_leaves': 27,

    'subsample': 0.68,
    'colsample_bytree': 0.63,
    'colsample_bylevel': 0.68,
    'colsample_bynode': 0.73,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.3,
    'reg_lambda': 3.3,
    'gamma': 2.5,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 33: MODERATE DEPTH + HIGH REG
# Balance clásico pero adaptado
# ==========================================
params_33_moderate_highreg = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.011,
    'n_estimators': 6500,

    'max_depth': 6,
    'min_child_weight': 27,
    'max_leaves': 33,

    'subsample': 0.72,
    'colsample_bytree': 0.67,
    'colsample_bylevel': 0.72,
    'colsample_bynode': 0.77,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 1.4,
    'reg_lambda': 3.6,
    'gamma': 2.6,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 34: CONSERVATIVE MANY FEATURES
# Muy conservador específicamente para 363 features
# ==========================================
params_34_conservative_363 = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.009,
    'n_estimators': 7500,

    'max_depth': 4,  # Shallow
    'min_child_weight': 40,  # Muy restrictivo
    'max_leaves': 20,

    'subsample': 0.65,
    'colsample_bytree': 0.58,
    'colsample_bylevel': 0.63,
    'colsample_bynode': 0.68,

    'scale_pos_weight': optimal_scale * 0.80,

    'reg_alpha': 2.2,
    'reg_lambda': 4.5,
    'gamma': 3.2,
    'max_delta_step': 1,
}

# ==========================================
# CONFIG 35: FOCUS ON NEW FEATURES
# Optimizado para aprovechar las nuevas features avanzadas
# ==========================================
params_35_new_features_focus = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'device': 'cuda',

    'learning_rate': 0.012,
    'n_estimators': 6000,

    'max_depth': 6,
    'min_child_weight': 26,
    'max_leaves': 35,

    # Permite más exploración de features
    'subsample': 0.72,
    'colsample_bytree': 0.68,
    'colsample_bylevel': 0.73,
    'colsample_bynode': 0.78,

    'scale_pos_weight': optimal_scale * 0.78,  # Ligeramente ajustado

    'reg_alpha': 1.3,
    'reg_lambda': 3.2,
    'gamma': 2.3,
    'max_delta_step': 1,
}

# ==========================================
# DICCIONARIO COMPLETO
# ==========================================
all_advanced_configs = {
    '26_hp_many_features': params_26_hp_many_features,
    '27_ultra_reg': params_27_ultra_reg,
    '28_deep_selection': params_28_deep_selection,
    '29_balanced_diverse': params_29_balanced_diverse,
    '30_extreme_sampling': params_30_extreme_sampling,
    '31_l1_dominant': params_31_l1_dominant,
    '32_adaptive': params_32_adaptive,
    '33_moderate_highreg': params_33_moderate_highreg,
    '34_conservative_363': params_34_conservative_363,
    '35_new_features_focus': params_35_new_features_focus,
}


In [40]:
results_advanced = {}

for i, j in all_advanced_configs.items():
    results_advanced[i] = train_and_evaluate(i, j,  X_train, y_train, X_val, y_val)


Entrenando: 26_hp_many_features
[0]	validation_0-auc:0.68004
[200]	validation_0-auc:0.75692
[400]	validation_0-auc:0.76896
[600]	validation_0-auc:0.77487
[800]	validation_0-auc:0.77844
[1000]	validation_0-auc:0.78081
[1200]	validation_0-auc:0.78270
[1400]	validation_0-auc:0.78414
[1600]	validation_0-auc:0.78516
[1800]	validation_0-auc:0.78580
[2000]	validation_0-auc:0.78641
[2200]	validation_0-auc:0.78686
[2400]	validation_0-auc:0.78727
[2600]	validation_0-auc:0.78753
[2800]	validation_0-auc:0.78775
[3000]	validation_0-auc:0.78799
[3200]	validation_0-auc:0.78815
[3400]	validation_0-auc:0.78817
[3600]	validation_0-auc:0.78827
[3800]	validation_0-auc:0.78817
[4000]	validation_0-auc:0.78816
[4005]	validation_0-auc:0.78818

RESULTADOS: 26_hp_many_features
AUC:       0.78830
Precision: 0.2232
Recall:    0.5841
FPR:       0.1785
Best iteration: 3505

Entrenando: 27_ultra_reg
[0]	validation_0-auc:0.69460
[200]	validation_0-auc:0.75456
[400]	validation_0-auc:0.76514
[600]	validation_0-auc:0.7

In [41]:
for i, j in results_advanced.items():
    print(i)
    print({k: j[k] for k in ['auc', 'precision', 'recall', 'fpr', 'best_iteration']})

26_hp_many_features
{'auc': 0.7882975355059794, 'precision': 0.22323146793934262, 'recall': 0.5840886203423967, 'fpr': 0.1784817291025505, 'best_iteration': 3505}
27_ultra_reg
{'auc': 0.7882245013620227, 'precision': 0.2222222222222222, 'recall': 0.591742195367573, 'fpr': 0.18187767519190634, 'best_iteration': 5000}
28_deep_selection
{'auc': 0.7885927517597536, 'precision': 0.23173553719008264, 'recall': 0.5647532729103726, 'fpr': 0.16442038982631152, 'best_iteration': 4747}
29_balanced_diverse
{'auc': 0.7883222993940712, 'precision': 0.2264910066266961, 'recall': 0.5782477341389728, 'fpr': 0.17342318440694754, 'best_iteration': 3970}
30_extreme_sampling
{'auc': 0.7876626729887521, 'precision': 0.2220549491908167, 'recall': 0.594159113796576, 'fpr': 0.18279741059110688, 'best_iteration': 3735}
31_l1_dominant
{'auc': 0.7881776275593166, 'precision': 0.22541462275169352, 'recall': 0.5830815709969789, 'fpr': 0.17595245675474902, 'best_iteration': 3526}
32_adaptive
{'auc': 0.78835150913303

In [42]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

cat_features = [
    col for col in X.columns
    if X[col].dtype == 'object'
]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []

for fold, (tr, va) in enumerate(skf.split(X, y), 1):

    model = CatBoostClassifier(
        iterations=4000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        early_stopping_rounds=300,
        verbose=200
    )

    model.fit(
        X.iloc[tr], y.iloc[tr],
        eval_set=(X.iloc[va], y.iloc[va]),
        cat_features=cat_features
    )

    preds = model.predict_proba(X.iloc[va])[:, 1]
    auc = roc_auc_score(y.iloc[va], preds)
    aucs.append(auc)

    print(f'Fold {fold} AUC: {auc:.5f}')

print(f'\nAUC medio: {np.mean(aucs):.5f} ± {np.std(aucs):.5f}')



0:	test: 0.5812597	best: 0.5812597 (0)	total: 228ms	remaining: 15m 10s
200:	test: 0.7626713	best: 0.7626713 (200)	total: 8.66s	remaining: 2m 43s
400:	test: 0.7703488	best: 0.7703488 (400)	total: 16.8s	remaining: 2m 31s
600:	test: 0.7740412	best: 0.7740514 (598)	total: 24.7s	remaining: 2m 19s
800:	test: 0.7762203	best: 0.7762203 (800)	total: 32.7s	remaining: 2m 10s
1000:	test: 0.7774847	best: 0.7774847 (1000)	total: 40.4s	remaining: 2m 1s
1200:	test: 0.7784942	best: 0.7784942 (1200)	total: 48.1s	remaining: 1m 52s
1400:	test: 0.7791412	best: 0.7791428 (1399)	total: 56s	remaining: 1m 43s
1600:	test: 0.7797442	best: 0.7797465 (1590)	total: 1m 3s	remaining: 1m 35s
1800:	test: 0.7803025	best: 0.7803069 (1798)	total: 1m 11s	remaining: 1m 27s
2000:	test: 0.7806039	best: 0.7806066 (1999)	total: 1m 19s	remaining: 1m 19s
2200:	test: 0.7809936	best: 0.7809936 (2200)	total: 1m 26s	remaining: 1m 11s
2400:	test: 0.7811228	best: 0.7811228 (2400)	total: 1m 34s	remaining: 1m 2s
2600:	test: 0.7812095	bes

In [43]:
print(aucs)

[0.7814695368196428, 0.7901325872009136, 0.7821804580344065, 0.7900942339340167, 0.783060356645839]


In [44]:
from sklearn.metrics import roc_auc_score, classification_report
import numpy as np

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#Threshold (solo para métricas de clasificación)
adj_threshold = 0.6

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)


FPR: 0.0004
Precision (PPV): 0.9459
AUC validación: 0.84605
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.95      0.07      0.13      4965

    accuracy                           0.92     61503
   macro avg       0.94      0.54      0.55     61503
weighted avg       0.93      0.92      0.89     61503



In [49]:
model = CatBoostClassifier(
        iterations=4000,
        learning_rate=0.03,
        depth=6,
        l2_leaf_reg=3,
        loss_function='Logloss',
        eval_metric='AUC',
        random_seed=42,
        early_stopping_rounds=300,
        verbose=200
    )

model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        cat_features=cat_features
    )

0:	test: 0.5834979	best: 0.5834979 (0)	total: 101ms	remaining: 6m 45s
200:	test: 0.7690315	best: 0.7690315 (200)	total: 8.74s	remaining: 2m 45s
400:	test: 0.7768406	best: 0.7768406 (400)	total: 21.4s	remaining: 3m 12s
600:	test: 0.7804207	best: 0.7804207 (600)	total: 29.7s	remaining: 2m 48s
800:	test: 0.7822303	best: 0.7822491 (798)	total: 37.7s	remaining: 2m 30s
1000:	test: 0.7833926	best: 0.7833932 (998)	total: 45.7s	remaining: 2m 16s
1200:	test: 0.7841934	best: 0.7841941 (1199)	total: 53.7s	remaining: 2m 5s
1400:	test: 0.7845520	best: 0.7845603 (1396)	total: 1m 1s	remaining: 1m 54s
1600:	test: 0.7850419	best: 0.7850685 (1596)	total: 1m 9s	remaining: 1m 44s
1800:	test: 0.7852859	best: 0.7853290 (1789)	total: 1m 17s	remaining: 1m 34s
2000:	test: 0.7857815	best: 0.7857874 (1999)	total: 1m 25s	remaining: 1m 25s
2200:	test: 0.7860093	best: 0.7860093 (2200)	total: 1m 33s	remaining: 1m 16s
2400:	test: 0.7864885	best: 0.7864885 (2400)	total: 1m 41s	remaining: 1m 7s
2600:	test: 0.7865636	bes

In [50]:
# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#Threshold (solo para métricas de clasificación)
adj_threshold = 0.6

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

FPR: 0.0013
Precision (PPV): 0.6373
AUC validación: 0.78771
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.64      0.03      0.05      4965

    accuracy                           0.92     61503
   macro avg       0.78      0.51      0.50     61503
weighted avg       0.90      0.92      0.89     61503

